In [1]:
#--------------------------------------------------------------------------------------------
#############################################################################################
#                             Beginning of actual code!
#############################################################################################
#--------------------------------------------------------------------------------------------

In [2]:
from itertools import product
from functools import reduce
from StateVect import StateVect
import Xoodoo
import math
from tqdm import tqdm   #Only for testing etc
import time             #Only for testing etc

In [3]:
#All Functions for execution will be listed here:

def activity(statevect):
    """
    Return the activity pattern of the state as a StateVect
    """
    pattern = StateVect(len(statevect)//3)
    for i in range(0, len(statevect), 3):
        for j in range(3):
            if statevect[i + j]:
                pattern[i//3] = 1
                break
    return pattern

def generate_basis(activity):
    """
    Return a list of StateVect that is a basis of the vector space defined by
    the activity pattern `activity`.
    """
    res = []
    l = len(3*activity)
    for i in range(len(activity)):
        if activity[i]:
            for j in range(3):
                new = StateVect(l)
                new[3*i + j] = 1
                res.append(new)
    return res

def affine(vec):
    """
    Creates basis and offset for an affine space based on a vector given as 
    input. Dimension will be twice the number of active S-Boxes
    """
    l = len(vec)
    act = activity(vec)
    basis = []
    off = StateVect(l)
    for i in range(l//3):
        if act[i] == 1:
            # Case 100
            if (vec[3*i] == 1 and vec[3*i+1] == 0 and vec[3*i+2] == 0):
                #Offset
                offNew = StateVect(l)
                offNew[3*i] = 1
                off = off | offNew
                #Basis vectors
                b1 = StateVect(l)
                b2 = StateVect(l)
                b1[3*i+1] = 1
                b2[3*i+2] = 1
                basis.append(b1)
                basis.append(b2)
                
            # Case 010
            if (vec[3*i] == 0 and vec[3*i+1] == 1 and vec[3*i+2] == 0):
                #Offset
                offNew = StateVect(l)
                offNew[3*i+1] = 1
                off = off | offNew
                #Basis vectors
                b1 = StateVect(l)
                b2 = StateVect(l)
                b1[3*i] = 1
                b2[3*i+2] = 1
                basis.append(b1)
                basis.append(b2)
                
            # Case 001
            if (vec[3*i] == 0 and vec[3*i+1] == 0 and vec[3*i+2] == 1):
                #Offset
                offNew = StateVect(l)
                offNew[3*i+2] = 1
                off = off | offNew
                #Basis vectors
                b1 = StateVect(l)
                b2 = StateVect(l)
                b1[3*i] = 1
                b2[3*i+1] = 1
                basis.append(b1)
                basis.append(b2)
                
            # Case 110
            if (vec[3*i] == 1 and vec[3*i+1] == 1 and vec[3*i+2] == 0):
                #Offset
                offNew = StateVect(l)
                offNew[3*i] = 1
                off = off | offNew
                #Basis vectors
                b1 = StateVect(l)
                b2 = StateVect(l)
                b1[3*i+2] = 1
                b2[3*i]   = 1
                b2[3*i+1] = 1
                basis.append(b1)
                basis.append(b2)
                
            # Case 101
            if (vec[3*i] == 1 and vec[3*i+1] == 0 and vec[3*i+2] == 1):
                #Offset
                offNew = StateVect(l)
                offNew[3*i+2] = 1
                off = off | offNew
                #Basis vectors
                b1 = StateVect(l)
                b2 = StateVect(l)
                b1[3*i+1] = 1
                b2[3*i]   = 1
                b2[3*i+2] = 1
                basis.append(b1)
                basis.append(b2)
            
            # Case 011
            if (vec[3*i] == 0 and vec[3*i+1] == 1 and vec[3*i+2] == 1):
                #Offset
                offNew = StateVect(l)
                offNew[3*i+1] = 1
                off = off | offNew
                #Basis vectors
                b1 = StateVect(l)
                b2 = StateVect(l)
                b1[3*i]   = 1
                b2[3*i+1] = 1
                b2[3*i+2] = 1
                basis.append(b1)
                basis.append(b2)
            
            # Case 111
            if (vec[3*i] == 1 and vec[3*i+1] == 1 and vec[3*i+2] == 1):
                #Offset
                offNew = StateVect(l)
                offNew[3*i] = 1
                off = off | offNew
                #Basis vectors
                b1 = StateVect(l)
                b2 = StateVect(l)
                b1[3*i+1] = 1
                b1[3*i+2] = 1
                b2[3*i]   = 1
                b2[3*i+2] = 1
                basis.append(b1)
                basis.append(b2)               
    return basis, off

def weight(vec):
    """
    Returns number of active bits in a vector
    """
    res = 0
    for i in range(len(vec)):
        if vec[i]:
            res += 1
    return res

def bigv(basis, v):
    """
    Increases weight of a vector from a space by adding suitable base vectors
    """
    w = weight(v)
    for i in range(len(basis)):
        if weight(v ^ basis[i]) > w:
            v = v ^ basis[i]
            w = weight(v)
    return v

def smallv(basis, v):
    """
    Decreases weight of a vector from a space by adding suitable base vectors
    """
    w = weight(v)
    for i in range(len(basis)):
        if weight(v ^ basis[i]) < w:
            v = v ^ basis[i]
            w = weight(v)
    return v

def act_of_basis(basis):
    """
    Computes the logical or of all basis vectors as new vector. Activity 
    pattern of result is returned
    """
    or_in = reduce((lambda p1, p2: p1 | p2), basis, StateVect(384))
    return activity(or_in)

def or_basis(basis):
    """
    Computes the loical or of all basis vectors and returns it as a StateVect
    """
    return reduce((lambda p1, p2: p1 | p2), basis, StateVect(384))
    
def rearrange_bases(b1, b2, v2):
    """
    Arrange vectors consistently in b1 and b2 according to passive
    S-boxes in b1. Keep track of original order by returning the list
    of the position of the S Box before rearranging
    """
    order = []
    res1 = []
    res2 = []
    res3 = StateVect(0)
    for i in range(len(b1)):
        res1.append(StateVect(size=0))
    for i in range(len(b2)):
        res2.append(StateVect(size=0))

    or1 = reduce((lambda p1, p2: p1 | p2), b1, StateVect(384))

    for i in range(128):
        # If active
        if any([or1[3*i + j] for j in range(3)]):
            for j in range(len(b1)):
                for k in range(3):
                    res1[j].append(b1[j][3*i + k])
            for j in range(len(b2)):
                for k in range(3):
                    res2[j].append(b2[j][3*i + k])
            for k in range(3):
                res3.append(v2[3*i+k])
            order.append(i)
        # If passive
        else:
            for j in range(len(b1)):
                for k in range(3, 0, -1):
                    res1[j].insert(0, b1[j][3*i + k - 1])
            for j in range(len(b2)):
                for k in range(3, 0, -1):
                    res2[j].insert(0, b2[j][3*i + k - 1])
            for k in range(3, 0, -1):
                res3.insert(0, v2[3*i+k-1])
            order.insert(0, i)

    return (res1, res2, res3, order)

def restore_bases(b1, b2, v2, order):
    """
    Change the order of the bases according to the order given.
    """
    res1 = []
    res2 = []
    res3 = StateVect(384)
    for i in range(len(b1)):
        res1.append(StateVect(384))
    for i in range(len(b2)):
        res2.append(StateVect(384))

    for i in range(128):
        for j in range(len(b1)):
            for k in range(3):
                res1[j][3*order[i] + k] = b1[j][3*i + k]
        for j in range(len(b2)):
            for k in range(3):
                res2[j][3*order[i] + k] = b2[j][3*i + k]
        for k in range(3):
            res3[3*order[i] + k] = v2[3*i + k]

    return (res1, res2, res3)

def indexOfActiveVector(basis, box):
    '''
    Gives back index of first vector active in S-Box provided
    '''
    for i in range(len(basis)):
        if(basis[i][3*box] == 1 or basis[i][3*box+1] == 1 or basis[i][3*box+2] == 1):
            return i
    return -1

def cutOut(a, i):
    '''
    Eliminates element at index i from array a
    '''
    if i == 0:
        return a[1:]
    else: 
        return a[:i-1] + a[i:]

def diffpair_is_valid(s1, s2):
    """
    Given two states, check if it is a possible 
    differential pair.
    """
    if activity(s1) != activity(s2):
        return False

    for i in range(0, len(s1), 3):
        sbox_in = s1[i:i + 3]
        sbox_out = s2[i:i + 3]
        if sbox_in != sbox_size*[0]:
            if not Xoodo.sbox_diff_is_valid(sbox_in, sbox_out):
                return False
    return True

def matching1v1(b1,v1,b2,v2):
    '''
    Tests two affine spaces against each other. Elements are 
    being tested for validity one by one
    '''
    for c1 in tqdm(product([0,1], repeat=len(b2))):
        tmp1 = v2
        for i in range(len(c1)):
            if c1[i]:
                tmp1 ^= b2[i]
        for c2 in product([0,1], repeat=len(b1)):
            tmp2 = v1
            for i in range(len(c2)):
                if c2[i]:
                    tmp2 ^= b1[i]
            if diffpair_is_valid(tmp1, tmp2):
                print("VALID HIT FOUND!!!!!!!!!")

def match(b1, v1, b2, v2):
    '''
    Recursivley divides the two bases of the affine spaces in
    a divide and conquer approach to limit search space
    '''
    if (len(b1) > 1 and len(b2) > 1):   
        #calculate activities
        or1 = reduce((lambda p1, p2: p1 | p2), b1, StateVect(384))
        or2 = reduce((lambda p1, p2: p1 | p2), b2, StateVect(384))
        actb1 = activity(or1)
        actb2 = activity(or2)
        actv1 = activity(v1)
        actv2 = activity(v2)

        #find first shared active vector S-Box
        for i in range(128):
            if(actb1[i] == 1 and actb2[i] == 1):
                break
        if i == 127:
            print("\n\ni = 127\n\n")
        i1 = indexOfActiveVector(b1, i)
        i2 = indexOfActiveVector(b2, i)

        add1 = b1[i1]
        add2 = b2[i2]
        b1 = cutOut(b1,i1)
        b2 = cutOut(b2,i2)
        match(b1, v1, b2, v2)
        match(b1, v1^add1, b2, v2^add2)

    else:
        #print("Now doing 1v1 reduction of affine spaces with dimensions " + str(len(b1)) + " and " + str(len(b2)))
        matching1v1(b1,v1,b2,v2)    
        
def rref(basis):
    """
    Take an arbitrary matrix of binary vector and put it into reduced row
    echelon form. Output the column index of the pivot for each line.
    """

    if len(basis) == 0:
        raise ValueError("Basis cannot be empty")

    line = 0
    pivots = []
    for i in range(len(basis[0])):
        j = line
        while j < len(basis) and not basis[j][i]:
            j += 1
        if j < len(basis):
            basis[j], basis[line] = basis[line], basis[j]
            for k in range(len(basis)):
                if basis[k][i] and k != line:
                    basis[k] ^= basis[line]
            line += 1
            pivots.append(i)

    return basis, pivots

def index_of_first_one(vec):
    """
    Returns index of first bit in vector that is of value '1'
    """
    for i in range(len(vec)):
        if vec[i]:
            break
    return i  

def xxx(basis, index):
    '''
    Returns first basis vector that has a one on given index
    '''
    for i in range(len(basis)):
        if basis[i][index] == 1:
            return basis[i]

def test_impr_clean_bases(vec, aff, off):
    """
    Removes vectors from affine/vector space bases that cannot match 
    through the non-linear layer
    """
    
    size = 0
    while size != len(vec + aff):
        
        #Check for improvment loop!
        size = len(vec + aff)
        
        #---------------------------------------------------------
        #--1--Checking for direct elimination based on offset-----
        #---------------------------------------------------------
        
        #Compute activites
        act_vec = act_of_basis(vec)
        or_aff = or_basis(aff)
        
        for i in range(384):
            if (act_vec[i//3] == 0 and or_aff[i] == 0 and off[i] == 1):
                return [], [], off
            
            
        #---------------------------------------------------    
        #--2--Checking for eliminations in Affine Space-----
        #---------------------------------------------------
      
        #printBasisTiny(vec)
        #printBasisTiny(aff)
        
        #Bring zeros from vec to the top
        ord_vec, ord_aff, ord_off, order = rearrange_bases(vec, aff, off)

        #rref affine to get isolated bits
        if(len(ord_aff)):
            ord_aff, pivots = rref(ord_aff)
        
        #calc new activities
        act_ord_vec = act_of_basis(ord_vec)
        act_ord_off = activity(ord_off)
        new_aff = []
        
        #iterate through pivots aka isolated bits aka interessting cases
        for i in range(len(pivots)):         
            if act_ord_vec[pivots[i]//3] == 1:
                new_aff.append(ord_aff[i])
            elif ord_off[pivots[i]] == 1:
                ord_off ^= xxx(ord_aff, pivots[i]) 
              
        #restore order from before
        vec, aff, off = restore_bases(ord_vec, new_aff, ord_off, order)

        #check if any of the bases are 0:
        if (len(vec)*len(aff)==0):
            break
            
        #---------------------------------------------------    
        #--3--Checking for eliminations in Vector Space-----
        #---------------------------------------------------    
        
        #Bring zeros from aff to the top
        ord_aff, ord_vec, ord_off, order = rearrange_bases(aff, vec, off)
        
        #rref to get isolated bits
        if len(ord_vec):
            ord_vec, pivots = rref(ord_vec)

        #calculate new activties
        act_ord_aff = act_of_basis(ord_aff)
        act_ord_off = activity(ord_off)
        new_vec = []
            
        #iterate through pivots aka isolated bits aka interesstin cases
        for i in range(len(pivots)):
            #aff = 0, off = 1 -> keep (improvment coudl be made by adding this to the offset of the vector space turning it into an affine one as well... But then everythign needs to be rewritten)
            if act_ord_aff[pivots[i]//3] == 1 or act_ord_off[pivots[i]//3] == 1:
                new_vec.append(ord_vec[i])
        
        #restore order from before
        aff, vec, off = restore_bases(ord_aff, new_vec, ord_off, order)
        
        #check if any of the bases are 0:
        if (len(vec)*len(aff)==0):
            break
        
        #---------------------------------------------------    
        #--4--Eliminations based on SBox behaviour-Vec------
        #---------------------------------------------------  
        #print("Step 3.1\n")
        vec, temp = rref(vec)
        aff, temp = rref(aff)
        or_vec = or_basis(vec)
        or_aff = or_basis(aff)
        act_vec = act_of_basis(vec)
        
        new_vec = []
        for v in vec:
            s = (index_of_first_one(v)//3)
            if (or_vec[3*s+0] == 1 and or_vec[3*s+1] == 0 and or_vec[3*s+2] == 0):
                if (or_aff[3*s+0] == 1 or off[3*s+0] == 1):
                    new_vec.append(v)
            elif (or_vec[3*s+0] == 0 and or_vec[3*s+1] == 1 and or_vec[3*s+2] == 0):
                if (or_aff[3*s+1] == 1 or off[3*s+1] == 1):
                    new_vec.append(v)
            elif (or_vec[3*s+0] == 0 and or_vec[3*s+1] == 0 and or_vec[3*s+2] == 1):
                if (or_aff[3*s+2] == 1 or off[3*s+2] == 1):
                    new_vec.append(v)
            else:
                new_vec.append(v)
        vec = new_vec
        
        #check if any of the bases are 0:
        if (len(vec)*len(aff)==0):
            break
        
        #---------------------------------------------------    
        #--5--Eliminations based on SBox behaviour-Aff------
        #---------------------------------------------------  
        #print("Step 3.2\n")
        vec, temp = rref(vec)
        aff, temp = rref(aff)
        or_vec = or_basis(vec)
        or_aff = or_basis(aff)
        
        new_aff = []
        for a in aff:
            s = (index_of_first_one(a)//3)
            if (or_aff[3*s+0] == 1 and or_aff[3*s+1] == 0 and or_aff[3*s+2] == 0):
                if (or_vec[3*s+0] == 1):
                    new_aff.append(a)
                elif(off[3*s+0] == 1):
                    off ^= v
            if (or_aff[3*s+0] == 0 and or_aff[3*s+1] == 1 and or_aff[3*s+2] == 0):
                if (or_vec[3*s+1] == 1):
                    new_aff.append(a)
                elif(off[3*s+1] == 1):
                    off ^= v
            if (or_aff[3*s+0] == 0 and or_aff[3*s+1] == 0 and or_aff[3*s+2] == 1):
                if (or_vec[3*s+2] == 1):
                    new_aff.append(a)
                elif(off[3*s+2] == 1):
                    off ^= v
        aff = new_aff    
        
        #check if any of the bases are 0:
        if (len(vec)*len(aff)==0):
            break
    return vec, aff, off

def quickCheck(vec, off):
    act_vec = act_of_basis(vec)
    act_off = activity(off)

    for i in range(128):
        if (act_vec[i] == 0 and act_off[i] == 1):
            return [], off
    return vec, off

def isSingleBitColumn(v, c):
    if (v[3*c] == 0 and v[3*c+1] == 0 and v[3*c+2] == 1) or (v[3*c] == 0 and v[3*c+1] == 1 and v[3*c+2] == 0) or (v[3*c] == 1 and v[3*c+1] == 0 and v[3*c+2] == 0):
        return True
    return False

def rearrangeVecFromOff(v1, o2):
    """

    """
    act_o2 = activity(o2)
    order = []
    res1 = []
    res3 = StateVect(0)
    for i in range(len(v1)):
        res1.append(StateVect(size=0))

    for i in range(128):
        # If active
        if act_o2[i]:
            for j in range(len(v1)):
                for k in range(3):
                    res1[j].append(v1[j][3*i + k])
            for k in range(3):
                res3.append(o2[3*i+k])
            order.append(i)  
        # If passive
        else:
            for j in range(len(v1)):
                for k in range(3, 0, -1):
                    res1[j].insert(0, v1[j][3*i + k - 1])
            for k in range(3, 0, -1):
                res3.insert(0, o2[3*i+k-1])
            order.insert(0, i)
    return (res1, res3, order)

def restoreVecOff(v1, o2, order):
    """
    
    """
    res1 = []
    res3 = StateVect(384)
    
    for i in range(len(v1)):
        res1.append(StateVect(384))

    for i in range(128):
        for j in range(len(v1)):
            for k in range(3):
                res1[j][3*order[i] + k] = v1[j][3*i + k]
        for k in range(3):
            res3[3*order[i] + k] = o2[3*i + k]

    return res1, res3

def numberOfActiveBits(b, index):
    c = 0
    for i in range(len(b)):
        if(b[i][index]):
            c += 1
    return c

#    Find isolated bits and eliminate if possible
def reduceVecOff(vec, off):
    check = 0
    while (check != len(vec)):
        check = len(vec)
        #Step 1: isolated bits
        vec, t = rref(vec)
        act_off = activity(off)
        new_vec = []
        for v in vec:
            elim = 0
            for i in range(384):
                if (v[i] == 1 and numberOfActiveBits(vec, i) == 1 and act_off[i//3] == 0): #aka isolated bit
                    elim = 1
            if(elim == 0):
                new_vec.append(v)
        vec = new_vec

        #Step 2: Single Bit Columns
        or_v = or_basis(vec)
        new_vec = []
        for v in vec:
            s = (index_of_first_one(v)//3)
            if (or_v[3*s+0] == 1 and or_v[3*s+1] == 0 and or_v[3*s+2] == 0):
                if (off[3*s+0] == 1):
                    new_vec.append(v)
            elif (or_v[3*s+0] == 0 and or_v[3*s+1] == 1 and or_v[3*s+2] == 0):
                if (off[3*s+1] == 1):
                    new_vec.append(v)
            elif (or_v[3*s+0] == 0 and or_v[3*s+1] == 0 and or_v[3*s+2] == 1):
                if (off[3*s+2] == 1):
                    new_vec.append(v)
            else:
                new_vec.append(v) 
        vec = new_vec
    return vec

def isSingleBitColumn(v, c):
    '''
    Returns TRUE is column "c" has exactly one active bit, 
    FALSE otherwise
    '''
    if (v[3*c] == 0 and v[3*c+1] == 0 and v[3*c+2] == 1) or (v[3*c] == 0 and v[3*c+1] == 1 and v[3*c+2] == 0) or (v[3*c] == 1 and v[3*c+1] == 0 and v[3*c+2] == 0):
        return True
    return False

def diffpair_is_valid(s1, s2):
    '''
    Checks whether a pair of vectors is compatible
    through non-linear layer chi
    '''
    for i in range(0, 384, 3):
        sbox_in = s1[i:i + 3]
        sbox_out = s2[i:i + 3]
        if sbox_in != 3*[0]:
            if not Xoodoo.sbox_diff_is_valid(sbox_in, sbox_out):
                return False
    return True    

def matchVecOff(vec, off):    
    '''
    Finds all matching vectors in vec that are compatible
    with off trhough non-linear layer chi
    '''
    #Temp:
    if len(vec) == 0:
        return 0
    
    #Step 1: Moving needed basis vectors to offset
    vec_off = StateVect(384)
    new_vec = []
    vec, pivots = rref(vec)
    for p in pivots:
        if(isSingleBitColumn(off, p//3) and off[p]):
            vec_off ^= xxx(vec,p)
        else:       
            new_vec.append(xxx(vec, p))
    vec = new_vec
    
    #Step 2: Actual checking
    hits = []
    act_off = activity(off)
    for choice in product([0, 1], repeat = len(vec)):
        t = vec_off
        for i in range(len(choice)):
            if choice[i]:
                t ^= vec[i]
        if act_off == activity(t):
            if diffpair_is_valid(t, off):
                hits.append([choice, t, off])
    return hits

def printOutputs(o):
    print("Number of trail cores found: " + str(len(o)) + "\n")
    for i in range(len(o)):
        print("\nTrail Core #" + str(i+1) + "\nu':")
        printVector(o[i][0])


In [4]:
#helpful function for debugging etc.
#These are all only being used during development etc. 
#and can be deleted once everything is done!
def printBasis(base):
    s = "-------------------------------------------------------------------------------------------------\nBase is now of dimension " + str(len(base)) + ":\n"
    for i in range(len(base)):
        s = s + "| "
        for j in range(len(base[0])//3):
            if (base[i][3*j] == 0 and base[i][3*j+1] == 0 and base[i][3*j+2] == 0):
                s = s + " 0  | "
            else:
                s = s + str(base[i][3*j]) +  str(base[i][3*j+1]) + str(base[i][3*j+2]) + " | "
        s = s+"\n"
    print(s + "-------------------------------------------------------------------------------------------------")
    
def printBasisTiny(base):
    s = s = "-------------------------------------------------------------------------------------------------------------------------------\nBase is now of dimension " + str(len(base)) + ":\n"
    for i in range(len(base)):
        for j in range(len(base[0])//3):
            if (base[i][3*j] == 1):
                s = s + "1|"
            elif(base[i][3*j+1]==1):
                s = s + "2|"
            elif(base[i][3*j+2]==1):
                s = s + "4|"
            else:
                s = s + " |"
        s = s+"\n"
    print(s + "-------------------------------------------------------------------------------------------------------------------------------")
    

    
def printVectorTiny(v):
    s = "-------------------------------------------------------------------------------------------------\nVector:\n"
    for j in range(len(v)//3):
        if (v[3*j] == 0 and v[3*j+1] == 0 and v[3*j+2] == 0):
            s = s + " |"
        else:
            s = s + "x|"
    s = s+"\n"
    print(s + "-------------------------------------------------------------------------------------------------")
    
def printVectorTinyCompact(v):
    s = "-------------------------------------------------------------------------------------------------\nVector:\n"
    for j in range(len(v)//3):
        if (v[3*j] == 0 and v[3*j+1] == 0 and v[3*j+2] == 0):
            s = s + " |"
        else:
            s = s + str(4*int(v[3*j+2]) + 2 * int(v[3*j+1]) + int(v[3*j])) + "|"
    s = s+"\n"
    print(s + "-------------------------------------------------------------------------------------------------")
    
    
def printActivity(v):
    s = "-------------------------------------------------------------------------------------------------\nActivity:\n"
    s = s + "|  "
    for j in range(len(v)):
        s = s + str(v[j]) + "  |  "
    s = s+"\n"
    print(s + "-------------------------------------------------------------------------------------------------")
    
def printActivityTiny(v):
    s = "-------------------------------------------------------------------------------------------------\nActivity:\n"
    for i in range(len(v)):
        if v[i] == 1:
            s += "x|"
        else:
            s += " |"
    s = s+"\n"
    print(s + "-------------------------------------------------------------------------------------------------")
    
def printVector(v):
    s = "-------------------------------------------------------------------------------------------------\nVector:\n"
    s = s + "| "
    for j in range(len(v)//3):
        if (v[3*j] == 0 and v[3*j+1] == 0 and v[3*j+2] == 0):
            s = s + " 0  | "
        else:
            s = s + str(v[3*j]) + str(v[3*j+1]) + str(v[3*j+2]) + " | "
    s = s+"\n-------------------------------------------------------------------------------------------------"
    s = s[0:298] + "\n" + s[298:490] + "\n" + s[490:682] + "\n" + s[682:]
    print(s)    
    
def timeToString(time):
    s = round(time%60)
    m = round(((time-s)/60)%60)
    h = round((((time-s-60*m)/3600)))
    return str(h) + ":" + str(m) + ":" + str(s)

In [5]:
############################################
##########---Main Code :))---###############
############################################

diff_trails = []


#Reading in of data 
with open("weight 80 cores.txt", 'r') as f:  #Change Path here to read in the trail core **PATH**
    l = f.readline()[:-1]
    while l:
        dt = Xoodoo.parse_xooloots_trail(l)
        dt[1][0].inv_linear()
        diff_trails.append(dt)
        l = f.readline()[:-1]           
dt = diff_trails[-1] 

Checke #1
Checke #2
Checke #3
{'type': 'DTc', 'tot_weight': 80, 'len': 4, 'part_weights': [32, 24, 16, 8], 'nb_states': 3}
{'type': 'DTc', 'tot_weight': 80, 'len': 4, 'part_weights': [32, 24, 16, 8], 'nb_states': 3}
{'type': 'DTc', 'tot_weight': 80, 'len': 4, 'part_weights': [32, 24, 16, 8], 'nb_states': 3}
{'type': 'DTc', 'tot_weight': 80, 'len': 4, 'part_weights': [32, 24, 16, 8], 'nb_states': 3}
{'type': 'DTc', 'tot_weight': 80, 'len': 4, 'part_weights': [32, 24, 16, 8], 'nb_states': 3}
{'type': 'DTc', 'tot_weight': 80, 'len': 4, 'part_weights': [32, 24, 16, 8], 'nb_states': 3}
{'type': 'DTc', 'tot_weight': 80, 'len': 4, 'part_weights': [32, 24, 16, 8], 'nb_states': 3}
{'type': 'DTc', 'tot_weight': 80, 'len': 4, 'part_weights': [32, 24, 16, 8], 'nb_states': 3}
{'type': 'DTc', 'tot_weight': 80, 'len': 4, 'part_weights': [32, 24, 16, 8], 'nb_states': 3}
{'type': 'DTc', 'tot_weight': 80, 'len': 4, 'part_weights': [32, 24, 16, 8], 'nb_states': 3}
{'type': 'DTc', 'tot_weight': 80, 'len':

In [6]:
dt = diff_trails[0]
b0 = dt[1][0].to_vect()
a4 = dt[1][2].to_vect()

In [7]:
"""
Main analysis part, Forward Analysis
"""
#Start Timer
start_time = time.time()

#--create W'/W from activity--
act_out = activity(a4)
Wprime = generate_basis(act_out)
print("Dimension of Vetor Space W and W': " + str(len(Wprime)) + "\n")

results = []
outputs = []

#--create U/U' from activity--
act_in = activity(b0)
U = generate_basis(act_in)
Uprime = []
for vec in U:
    Uprime.append(Xoodoo.to_state(vec).linear().to_vect())
print("Dimension of Vetor Space U and U': " + str(len(U)) + "\n")   

for choice in tqdm(product([0,1], repeat=len(Wprime))):
    #--create V'/V from a picked w€W-
    wprime = StateVect(384)
    for i in range(len(choice)):
        if choice[i]:
            wprime ^= Wprime[i]

    if activity(wprime) == act_out: 
        #--constructing V and V'--
        w = Xoodoo.to_state(wprime).inv_linear().to_vect()
        Vprime, Vprimeoff = affine(w)
        Voff = Xoodoo.to_state(Vprimeoff).inv_linear().to_vect()
        V = []
        for vec in Vprime:
            V.append(Xoodoo.to_state(vec).inv_linear().to_vect())

        #--Thin out U' and V against each other--
        thinUprime, thinV, thinVoff = test_impr_clean_bases(Uprime, V, Voff)

        #--Extra elimination in case of Vector Space and offset
        if(len(thinUprime) > 0 and len(thinV) == 0):
            thinUprime, thinVoff, order = rearrangeVecFromOff(thinUprime, thinVoff)
            thinUprime = reduceVecOff(thinUprime, thinVoff)
            thinUprime, thinVoff = restoreVecOff(thinUprime, thinVoff, order)       
            thinUprime, thinVoff = quickCheck(thinUprime, thinVoff)
            
        #--keep track of results--            
        results.append([choice, len(thinUprime), len(thinV)])
        if len(thinUprime)!= 0:
            o = matchVecOff(thinUprime, thinVoff)
            for i in range(len(o)):
                outputs.append([o[i][1], o[i][2]])

duration = time.time() - start_time
print("# dim(U) = " + str(len(U)) + "\t --- dim(W) = " + str(len(Wprime)) + "\t --- time: " + timeToString(duration))
print("\n")
print("number of computed attempts = " + str(len(results)) + "\n\n")

printOutputs(outputs)

114it [00:00, 1139.90it/s]

Dimension of Vetor Space W and W': 12

Dimension of Vetor Space U and U': 48



4096it [1:34:52,  1.39s/it]

# dim(U) = 48	 --- dim(W) = 12	 --- time: 1:34:53


number of computed attempts = 2401
To see all use print(results)


Number of trail cores found: 2


Trail Core #1
u':
-------------------------------------------------------------------------------------------------
Vector:
| 110 | 010 | 110 |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  
| 110 | 100 | 110 |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  
| 110 | 010 | 110 |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  
| 110 | 100 | 110 |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |